<a href="https://colab.research.google.com/github/leolaipelt/EarthEngine-Python-Tutorial/blob/master/10_Incendios_na_Amazonia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Incêndios na Amazônia - 2019

Seguindo com exemplos do que podemos fazer com o EarthEngine, existe um produto da NASA, em relação a incêndios, disponibilizado no banco de dados do EE. O *Fire Information for Resource Management System (FIRMS)* contém uma máscara raster com informações sobre incêndios no mundo todo.  O produto é elaborado utilizando MODIS MOD14/MYD14 *Fire and Thermal Anomalies*. 

* Resolução 1000m.
* Bandas: T21 (Temperatura de Brilho).
* Range: 300-509.29 Kelvin.

In [0]:
!pip install earthengine-api

In [0]:
import ee; ee.Authenticate()

In [0]:
ee.Initialize()

##Nossa Funcao de retirar nuvens das imagens do L8
def Cloud_MASKL8(image):
    
    quality = image.select('pixel_qa');
    c01 = quality.eq(322); #//Clear, low confidence cloud
    c02 = quality.eq(324); #//water, low confidence cloud
    c03 = quality.eq(1346);# //Clear terrain, terrain occluded
    mask = c01.Or(c02).Or(c03);
    return image.updateMask(mask);

L8=(ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
      .filterDate("2017-08-01","2019-08-30")).map(Cloud_MASKL8).mosaic() #Aqui criamos um mosaico com as imagens disponiveis de L8 no periodo que estipulamos

amazonia = ee.FeatureCollection("users/leolaipelt/AmazoniaLegal") ##Um SHP da Amazonia

dataset = ee.ImageCollection('FIRMS').filter(ee.Filter.date('2019-08-01', '2019-08-26')) ##Aqui selecionamos as imagens do FIRMS
 
fire = dataset.select('T21').max(); #Queremos apenas os valores maximos encontrado nesse periodo para aquele pixel
 


In [0]:
#Vamos visualizar esse produto com o mapa que já criamos!
import folium

def add_ee_layer(self,imagem,visParams, nome):
  mapID = ee.Image(imagem).getMapId(visParams)
  folium.raster_layers.TileLayer(
    tiles ="https://earthengine.googleapis.com/map/"+mapID['mapid']+
      "/{z}/{x}/{y}?token="+mapID['token'],
    attr = "Nosso Mapinha do Curso de Python",
    name = nome,
    overlay = True,
    control = True
  ).add_to(self)
  
meuMap = folium.Map(location=[20, 0], zoom_start=3, height=800)

folium.Map.add_ee_layer = add_ee_layer

In [34]:
# Adicionando no mapa a imagem que criamos
meuMap.add_ee_layer(L8.clip(amazonia), {'bands':['B4','B3','B2'],'min':0,'max':3000,'gamma':1.5}, 'Amazonia RGB')
meuMap.add_ee_layer(fire.clip(amazonia), {'min':300,'max':400,'palette':['red', 'orange', 'yellow']}, 'Amazonia Incêndios')

# Adicionando um controlador de layers no nosso mapa
meuMap.add_child(folium.LayerControl())

#Ver nosso mapa!
display(meuMap)

[Ref: GeoBlog](https://mygeoblog.com/)